In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
#input_file = "output_20191008_0336AM.csv" #complete
#input_file = "output_20191116_1256PM.csv" #partial
input_file = "output_20200219_2125PM.csv" #double
df = pd.read_csv(input_file, header = 0)

In [5]:
df['FLOW'] = df['SRCNODE']+'-'+df['DESTNODE']
df['FM']=df['FAILURE']+df['MISSING']

In [6]:
df_failure = df[ (df.FM==1)]

In [7]:
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import cross_val_score
from sklearn.multiclass import OneVsOneClassifier
from sklearn.calibration import CalibratedClassifierCV

In [8]:
df_dummy = pd.get_dummies(df_failure[['SRCNODE', 'DESTNODE','SRC_R','DEST_S','FM','RETRIES','FILESIZE', 'THROUGHPUT','LABEL']], prefix=['SRC', 'DEST','SRC_R','DEST_S'], columns=['SRCNODE', 'DESTNODE','SRC_R','DEST_S'])


In [9]:
df_dummy['THROUGHPUT']=df_dummy['THROUGHPUT'].str[0:-4]
df_dummy['THROUGHPUT']=pd.to_numeric(df_dummy['THROUGHPUT'], errors='coerce')
df_dummy = df_dummy.replace(np.nan, 0, regex=True)
y_complete=df_dummy['LABEL']
X_complete=df_dummy.drop(['LABEL'],axis=1)

In [10]:
df_dummy_cate = pd.get_dummies(df_failure[['SRCNODE', 'DESTNODE','SRC_R','DEST_S','FM','RETRIES','LABEL']], prefix=['SRC', 'DEST','SRC_R','DEST_S'], columns=['SRCNODE', 'DESTNODE','SRC_R','DEST_S'])
y_cate=df_dummy_cate['LABEL']
X_cate=df_dummy_cate.drop(['LABEL'],axis=1)
X_cate[0:1]

,FM,RETRIES,SRC_CacheCal,SRC_CacheFNAL,SRC_CacheNY,SRC_CacheSDSC,SRC_CacheUC,SRC_CacheUNL,SRC_OriginCal,SRC_OriginFNAL,...,SRC_R_OriginUNLRouter,DEST_S_CacheCENICRouter,DEST_S_CacheNYRouter,DEST_S_CacheStarLightRouter,DEST_S_CacheUNLRouter,DEST_S_OriginCalRouter,DEST_S_OriginNCSARouter,DEST_S_OriginSDSCRouter,DEST_S_OriginStarLightRouter,DEST_S_OriginUNLRouter
8,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import multilabel_confusion_matrix,balanced_accuracy_score,f1_score

In [12]:
def accuracy(model, test_data, classes, k):
    if(k==0): return null,null
    num = classes.size
    correct_class={}
    correct=0
    for label in classes:
        label_pred=[]
        isCorrect=False
        test_data_1=test_data[test_data.LABEL==label]
        if test_data_1.shape[0]==0:
            print("No match")
            continue
        x_test_1=test_data_1.drop(['LABEL'],axis=1)
        test_prob_1 = model.predict_proba(x_test_1)
        label_array=test_prob_1.mean(axis=0)
        #label_array=np.nanmean(np.where(test_rf_prob_1!=0,test_rf_prob_1,np.nan),0)
        label_index = label_array.argmax()
        label_index_sort = label_array.argsort()
        #label_pred[0] = classes[label_index]
        for j in range(0,k): 
            if(j==0):
                label_pred.append(classes[label_index])
            else:
                label_pred.append(classes[label_index_sort[(-1)*j-1]])
            #print(j,label_pred[j],label)
            if(label==label_pred[j]):
                isCorrect=True
                break
               
        #print(label_pred)
        #print(isCorrect)
        if isCorrect:
            correct=correct+1
            correct_class[label]=1
        else:
            correct_class[label]=0
    return correct_class, correct, correct/num 

In [13]:
def print_s_c(c):
    s_c=sorted(c.items(), key=lambda x: x[1], reverse=True)
    num_link=0;num_host=0;link_pos=0;host_pos=0
    for i in s_c:
        name=i[0]
        flag=i[1]
        if 'Link' in name:
            num_link+=1
            if flag==1:
                link_pos+=1
        else:
            num_host+=1
            if flag==1:
                host_pos+=1
    return link_pos*1.0/num_link,host_pos*1.0/num_host

In [14]:
training_dataset={"complete":{"X":X_complete,
                 "y":y_complete},
                 "cate":{"X":X_cate,
                 "y":y_cate}}

In [15]:
def train_dt(classifier):
    df=[df_dummy,df_dummy_cate]
    i=0
    for k, d in training_dataset.items():
        X=d["X"]
        y=d['y']
        clf = classifier
        clf_model=clf.fit(X,y)
        pred=clf.predict(X)
        balanced_accu=balanced_accuracy_score(y, pred)
        f1=f1_score(y, pred,average='weighted')
        print(str(k)+":balanced_accu="+str(balanced_accu)+":F1-Score="+str(f1))
        for j in range(1,4):
            c,correct,accu=accuracy(clf_model, df[i], clf.classes_,j)
            print("Top-"+str(j)+" Accu="+str(accu)) 
        i=i+1


In [16]:
linear_svc=svm.LinearSVC(random_state=0)
cal_linear=CalibratedClassifierCV(base_estimator=linear_svc,cv=2)
cal_svc = make_pipeline(StandardScaler(), CalibratedClassifierCV(base_estimator=SVC(gamma='auto'),cv=2))
#ovo_lin_model=OneVsOneClassifier(svm.LinearSVC(max_iter=10000,random_state=0))
train_dt(cal_svc)

complete:balanced_accu=0.16962787115708491:F1-Score=0.23735364593703434
Top-1 Accu=0.36363636363636365
Top-2 Accu=0.6060606060606061
Top-3 Accu=0.6818181818181818
cate:balanced_accu=0.15699045028414643:F1-Score=0.20294405587232228
Top-1 Accu=0.3181818181818182
Top-2 Accu=0.5454545454545454
Top-3 Accu=0.6818181818181818


In [71]:
%%time
linear_svc=svm.LinearSVC(random_state=0)
clf = make_pipeline(StandardScaler(), CalibratedClassifierCV(base_estimator=SVC(gamma='auto'),cv=2))
#clf=CalibratedClassifierCV(base_estimator=svc,cv=2)
clf_model=clf.fit(X_complete,y_complete)

CPU times: user 1min 24s, sys: 672 ms, total: 1min 25s
Wall time: 1min 15s


In [13]:
c,r=accuracy(cal_clf, df_dummy, ovo_lin_model.classes_,1)
c,r

(40, 0.6060606060606061)

In [14]:
from sklearn.metrics import multilabel_confusion_matrix,balanced_accuracy_score,f1_score
y_pred_rf = cal_clf.predict(x)
print(balanced_accuracy_score(y, y_pred_rf))
f1_score(y, y_pred_rf,average='weighted')

0.2463278028318829


/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.3680773409448971

In [15]:
c,r=accuracy(lin_df, df_dummy, ovo_lin_model.classes_,10)
c,r

AttributeError: 'numpy.ndarray' object has no attribute 'predict_proba'

In [16]:
ovo_svm_model=OneVsOneClassifier(svm.SVC(random_state=0)).fit(x, y)
#svm_pred=svm_model.predict(x)
svm_df = ovo_svm_model.decision_function(x)

/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/

/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/

/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/

/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/

/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/

...

/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/

/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/

/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/

/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/

/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/

In [17]:
def accuracy_ori(model, test_data, classes):
    num = classes.size
    correct=0
    for label in classes:
        test_data_1=test_data[test_data.LABEL==label]
        x_test_1=test_data_1.drop(['LABEL'],axis=1)
        test_prob_1 = model.decision_function(x_test_1)
        label_index=test_prob_1.sum(axis=0).argmax()
        label_pred = classes[label_index]
        if(label==label_pred):
            correct=correct+1
    return correct, correct/num

In [18]:
c,r=accuracy_ori(lin_df, df_dummy, ovo_lin_model.classes_)
c,r

AttributeError: 'numpy.ndarray' object has no attribute 'decision_function'

In [19]:
from sklearn.metrics import multilabel_confusion_matrix,balanced_accuracy_score,f1_score

In [20]:
multilabel_confusion_matrix(y, svm_pred)

NameError: name 'svm_pred' is not defined

In [ ]:
cross_val_score(clf_svm, x, y, scoring='accuracy',cv=5)

In [ ]:
cross_val_score(clf_linsvm, x, y, scoring='accuracy',cv=5)

In [ ]:
f1_score(y, lin_pred,average='weighted')

In [ ]:
f1_score(y, svm_pred,average='weighted')

In [ ]:
clf_svm.score(x,y)

In [ ]:
clf_linsvm.score(x,y)

In [ ]:
balanced_accuracy_score(y, svm_pred)

In [ ]:
balanced_accuracy_score(y, lin_pred)

In [ ]:
x.values.shape

In [ ]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import SVC

In [ ]:
classifier = BinaryRelevance(classifier = SVC(probability=True),
    require_dense = [False, True])

classifier.fit(x.to_numpy(), y.to_numpy())

In [21]:
test_file = "output_20191008_0336AM.csv" #complete
#test_file = "big.csv" #partial
test_df = pd.read_csv(test_file, header = 0)
test_df['FLOW'] = test_df['SRCNODE']+'-'+test_df['DESTNODE']
test_df['FM']=test_df['FAILURE']+test_df['MISSING']
test_df_failure = test_df[ (test_df.FM==1)]

test_df_dummy = pd.get_dummies(test_df_failure[['SRCNODE', 'DESTNODE','FM','RETRIES','LABEL']], prefix=['SRC', 'DEST'], columns=['SRCNODE', 'DESTNODE'])
test_y=test_df_failure['LABEL']
test_x=test_df_dummy.drop(['LABEL'],axis=1)

In [25]:
%%time
c,r=accuracy(cal_clf, test_df_dummy, cal_clf.classes_,3)
c,r

No match
No match
No match
No match
No match
CPU times: user 1.4 s, sys: 44.4 ms, total: 1.44 s
Wall time: 373 ms


(36, 0.5454545454545454)